In [2]:
library(dataiku)
dkuSourceLibR("pwr_analysis.R")

# Change to the ff if running on your local machine:
# source("pwr_analysis.R") 

Warning message:
“package ‘dataiku’ was built under R version 4.1.3”


[1] "SEARCHING FOR "
[1] "pwr_analysis.R"
[1] "IN"
[1] "/data/dataiku/dss_data/jupyter-run/dku-workdirs/TEST_26/Power_Analysis_Demo_Notebooka4c6978a-dssuser_donatpat/project-r-src/TEST_26/R"
[1] "Loading source file /data/dataiku/dss_data/jupyter-run/dku-workdirs/TEST_26/Power_Analysis_Demo_Notebooka4c6978a-dssuser_donatpat/project-r-src/TEST_26/R/pwr_analysis.R"


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


NULL

# Demo for Apriori Power Analysis

**Objective**
- Apriori power analysis is conducted before data collection to determine the required sample size needed to achieve a desired level of power, typically 80% or 90%. It ensures that the study is adequately powered to detect a meaningful effect.
- Unlike post hoc power analysis, which evaluates power after a study is conducted, apriori analysis is prospective and used for study planning.

**When to use it?**
- When designing a study to determine the necessary sample size.
- When ensuring that the study has a high probability of detecting a real effect, reducing the risk of Type II errors (false negatives).

**What are you testing?** 

This study uses **One-Sample Proportion Test** with the following null and alternative hypotheses:

- **Null Hypothesis:** The LLM's performance performs worse than the baseline PCC by a meaningful margin.  
  $$
  H_0: p \leq p_{\text{PCC}}
  $$
  where $p_{\text{PCC}}$ is the expected accuracy of a random classifier given the outcome distribution.  

- **Alternative Hypothesis** The LLM's performance exceeds the PCC by a meaningful margin.  
  $$
  H_A: p >> p_{\text{PCC}}
  $$
  
**What do you need?**   

1. **Significance Level** ($\alpha$) – The probability of making a Type I error (commonly set at 0.05). However, since we simultaneously testing multiple hypotheses, we instead set FWER $\alpha$ to 0.05. The significance level for each test $\alpha'$ is then set to $\alpha' = \frac{0.05}{m}$.
2. **Statistical Power** ($1-\beta$) – The probability of correctly rejecting the null hypothesis (commonly set at 80% or 90%).
3. **Effect Size** – The effect size is calculated as the Cohen's $h$ between $p$ and $p_{PCC}$. However, to establish meaningful improvement, we are only interested in $p$ that would generate the desired improvement of $ p_{\text{Target}} = 1.25 \times p_{\text{PCC}}$ for balanced outcomes, and of at least $p_{\text{Target}} = 1.15 \times p_{\text{PCC}}$ for imbalanced outcomes.
4. **Test Type** – One-tailed or two-tailed test.

**How to code it?**

`pwr.p.test(sig.level = alpha, power = power, h = h, alternative = "greater")`

1. `sig.level` is the **Significance Level**
2. `power` is the **Statistical Power**
3. `h` is the **Effect Size**, calculated as `ES.h(p_target, pcc)` 
4. `alternative` is the **Test Type**, `greater` since we are interested in gathering evidence to support $H_A: p >> p_{\text{PCC}}$.

## Create reference dataframe

In [3]:
# Create the dataframe
df <- data.frame(
    name = c(
        'Interviewer Challenges?',
        'Level of Challenge by Interviewer',
        'Israeli Perspective',
        'Palestinian Perspective',
        'Indiv Humanization Level',
        'Is Hard Ride',
        'Final Rating',
        'Casualties - Body',
        'Victim Sympathy',
        'Justification',
        'Challenge to justification',
        'Casualties - 5 sentence',
        'War Crimes Allegation Presence?',
        'War Crimes Perpetrator Mentioned?'
    ),
    cat_1 = c(3491, 942, 2340, 217, 1085, 1467, 458, 2765, 721, 741, 322, 752, 436, 375),
    cat_2 = c(4983, 2549, 6134, 8257, 7389, 7007, 1009, 5687, 1411, 1394, 419, 7702, 622, 684)
)
                  
df

name,cat_1,cat_2
<chr>,<dbl>,<dbl>
Interviewer Challenges?,3491,4983
Level of Challenge by Interviewer,942,2549
Israeli Perspective,2340,6134
Palestinian Perspective,217,8257
Indiv Humanization Level,1085,7389
Is Hard Ride,1467,7007
Final Rating,458,1009
Casualties - Body,2765,5687
Victim Sympathy,721,1411


## Control FWER

In [4]:
alpha = 0.05
no_of_tests <- 20
fwer_alpha <- alpha/no_of_tests

## Sample Size Calculation

In [5]:
var = 'War Crimes Allegation Presence?'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
range_accs <- c(0.80, 0.85, 0.90, 0.95, 0.97)

calculate_n_across_accs(name=var, cat_counts=cat_counts, accs=range_accs, alpha=fwer_alpha)

name,cat_1,cat_2,distribution,multiplier,pcc,min_target_acc,desired_accs,n
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
War Crimes Allegation Presence?,436,622,0.5879017,1.25,0.5154534,0.6443168,0.6443168,1942
War Crimes Allegation Presence?,436,622,0.5879017,1.25,0.5154534,0.6443168,0.8000000,355
War Crimes Allegation Presence?,436,622,0.5879017,1.25,0.5154534,0.6443168,0.8500000,241
War Crimes Allegation Presence?,436,622,0.5879017,1.25,0.5154534,0.6443168,0.9000000,166
War Crimes Allegation Presence?,436,622,0.5879017,1.25,0.5154534,0.6443168,0.9500000,113
War Crimes Allegation Presence?,436,622,0.5879017,1.25,0.5154534,0.6443168,0.9700000,94


In [6]:
var = 'War Crimes Perpetrator Mentioned?'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
range_accs <- c(0.80, 0.85, 0.90, 0.95, 0.97)

calculate_n_across_accs(name=var, cat_counts=cat_counts, accs=range_accs, alpha=fwer_alpha)

name,cat_1,cat_2,distribution,multiplier,pcc,min_target_acc,desired_accs,n
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
War Crimes Perpetrator Mentioned?,375,684,0.6458924,1.25,0.5425692,0.6782114,0.6782114,1708
War Crimes Perpetrator Mentioned?,375,684,0.6458924,1.25,0.5425692,0.6782114,0.8000000,428
War Crimes Perpetrator Mentioned?,375,684,0.6458924,1.25,0.5425692,0.6782114,0.8500000,280
War Crimes Perpetrator Mentioned?,375,684,0.6458924,1.25,0.5425692,0.6782114,0.9000000,188
War Crimes Perpetrator Mentioned?,375,684,0.6458924,1.25,0.5425692,0.6782114,0.9500000,125
War Crimes Perpetrator Mentioned?,375,684,0.6458924,1.25,0.5425692,0.6782114,0.9700000,103


In [7]:
var <- 'Palestinian Perspective'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
range_accs <- c(0.80, 0.85, 0.90, 0.95, 0.97)

calculate_n_across_accs(name=var, cat_counts=cat_counts, accs=range_accs, alpha=fwer_alpha)

Outcome is too imbalanced; minimum target accuracy is too high: 109.26%



# Demo for Post-Hoc Power Analysis

In [8]:
library(xlsx)
library(dataiku)
library(dplyr)

## Gather results

In [9]:
sheet_name <- "1. Victim Sympathy"
content <- dkuManagedFolderDownloadPath("m74dUoOQ","human_validation.xlsx", as="raw")
temp_f=tempfile()
writeBin(object=content, con=temp_f)
df_res <- read.xlsx(temp_f, sheetName=sheet_name)

## Power Calculation

In [10]:
# Get PCC from apriori power analysis
var = 'Victim Sympathy'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
pcc <- calculate_pcc(cat_counts)

# Get p from observed model accuracy
observed_acc <- mean(df_res$match)

# Check power
check_posthoc_power(pcc, observed_acc)

[1] "Model performs 172.35% better than baseline PCC at controlled FWER alpha = 0.05 and at least the standard 80% power."


# DRAFT

In [0]:
# Get PCC from apriori power analysis
var = 'Victim Sympathy'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
pcc <- calculate_pcc(cat_counts)

# Get p from observed model accuracy
sheet_name <- "1. Victim Sympathy"
content <- dkuManagedFolderDownloadPath("m74dUoOQ","human_validation.xlsx", as="raw")
temp_f=tempfile()
writeBin(object=content, con=temp_f)
df_res <- read.xlsx(temp_f, sheetName=sheet_name)
observed_acc <- mean(df_res$match)

# Check power if at least 80%
h <- calculate_cohens_h(observed_acc, pcc)
multiplier  <- round(observed_acc/pcc*100, 2)
power <- pwr.p.test(h = h, sig.level = fwer_alpha, alternative = "greater", n = nrow(df_res))$power
print(paste0('Model performs ', multiplier, '% better than PCC with ', power*100, '% power.'))

In [0]:
df_res %>% drop_na()

In [0]:
# Get PCC from apriori power analysis
var = 'Interviewer Challenges?'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
pcc <- calculate_pcc(cat_counts)

# Get p from observed model accuracy
sheet_name <- "2. Interviewer Challenges"
content <- dkuManagedFolderDownloadPath("m74dUoOQ","human_validation.xlsx", as="raw")
temp_f=tempfile()
writeBin(object=content, con=temp_f)
df_res <- read.xlsx(temp_f, sheetName=sheet_name)
colnames(df_res) <- c('analysis_id', 'ai_rating', 'human_rating', 'match')
observed_acc <- df_res %>% drop_na() %>% pull(match) %>% mean()

# Check power if at least 80%
h <- calculate_cohens_h(observed_acc, pcc)
multiplier  <- round(observed_acc/pcc*100, 2)
power <- pwr.p.test(h = h, sig.level = fwer_alpha, alternative = "greater", n = nrow(df_res))$power
print(paste0('Model performs ', multiplier, '% better than PCC with ', power*100, '% power.'))

In [0]:
# Get PCC from apriori power analysis
var = 'Israeli Perspective'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
pcc <- calculate_pcc(cat_counts)

# Get p from observed model accuracy
sheet_name <- "3. Israeli Perspective"
content <- dkuManagedFolderDownloadPath("m74dUoOQ","human_validation.xlsx", as="raw")
temp_f=tempfile()
writeBin(object=content, con=temp_f)
df_res <- read.xlsx(temp_f, sheetName=sheet_name)
colnames(df_res) <- c('analysis_id', 'ai_rating', 'human_rating', 'match')
observed_acc <- df_res %>% drop_na() %>% pull(match) %>% mean()

# Check power if at least 80%
h <- calculate_cohens_h(observed_acc, pcc)
multiplier  <- round(observed_acc/pcc*100, 2)
power <- pwr.p.test(h = h, sig.level = fwer_alpha, alternative = "greater", n = nrow(df_res))$power
print(paste0('Model performs ', multiplier, '% better than PCC with ', power*100, '% power.'))

In [0]:
# Get PCC from apriori power analysis
var = 'Palestinian Perspective'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
pcc <- calculate_pcc(cat_counts)

# Get p from observed model accuracy
sheet_name <- "4. Palestinian Perspective"
content <- dkuManagedFolderDownloadPath("m74dUoOQ","human_validation.xlsx", as="raw")
temp_f=tempfile()
writeBin(object=content, con=temp_f)
df_res <- read.xlsx(temp_f, sheetName=sheet_name)
colnames(df_res) <- c('analysis_id', 'ai_rating', 'human_rating', 'match')
observed_acc <- df_res %>% drop_na() %>% pull(match) %>% mean()

# Check power if at least 80%
h <- calculate_cohens_h(observed_acc, pcc)
multiplier  <- round(observed_acc/pcc*100, 2)
power <- pwr.p.test(h = h, sig.level = fwer_alpha, alternative = "greater", n = nrow(df_res))$power
print(paste0('Model performs ', multiplier, '% better than PCC with ', power*100, '% power.'))

In [0]:
# Get PCC from apriori power analysis
var = 'Indiv Humanization Level'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
pcc <- calculate_pcc(cat_counts)

# Get p from observed model accuracy
sheet_name <- "5. Indiv Humanization Level"
content <- dkuManagedFolderDownloadPath("m74dUoOQ","human_validation.xlsx", as="raw")
temp_f=tempfile()
writeBin(object=content, con=temp_f)
df_res <- read.xlsx(temp_f, sheetName=sheet_name)
colnames(df_res) <- c('analysis_id', 'ai_rating', 'human_rating', 'match')
observed_acc <- df_res %>% drop_na() %>% pull(match) %>% mean()

# Check power if at least 80%
h <- calculate_cohens_h(observed_acc, pcc)
multiplier  <- round(observed_acc/pcc*100, 2)
power <- pwr.p.test(h = h, sig.level = fwer_alpha, alternative = "greater", n = nrow(df_res))$power
print(paste0('Model performs ', multiplier, '% better than PCC with ', power*100, '% power.'))

In [0]:
# Get PCC from apriori power analysis
var = 'Is Hard Ride'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
pcc <- calculate_pcc(cat_counts)

# Get p from observed model accuracy
sheet_name <- "6. is Hard Ride"
content <- dkuManagedFolderDownloadPath("m74dUoOQ","human_validation.xlsx", as="raw")
temp_f=tempfile()
writeBin(object=content, con=temp_f)
df_res <- read.xlsx(temp_f, sheetName=sheet_name)
colnames(df_res) <- c('analysis_id', 'ai_rating', 'human_rating', 'match')
observed_acc <- df_res %>% drop_na() %>% pull(match) %>% mean()

# Check power if at least 80%
h <- calculate_cohens_h(observed_acc, pcc)
multiplier  <- round(observed_acc/pcc*100, 2)
power <- pwr.p.test(h = h, sig.level = fwer_alpha, alternative = "greater", n = nrow(df_res))$power
print(paste0('Model performs ', multiplier, '% better than PCC with ', power*100, '% power.'))

In [0]:
# Get PCC from apriori power analysis
var = 'Casualties - Body'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
pcc <- calculate_pcc(cat_counts)

# Get p from observed model accuracy
sheet_name <- "7. Casualties - Body"
content <- dkuManagedFolderDownloadPath("m74dUoOQ","human_validation.xlsx", as="raw")
temp_f=tempfile()
writeBin(object=content, con=temp_f)
df_res <- read.xlsx(temp_f, sheetName=sheet_name)
colnames(df_res) <- c('analysis_id', 'ai_rating', 'human_rating', 'match')
observed_acc <- df_res %>% drop_na() %>% pull(match) %>% mean()

# Check power if at least 80%
h <- calculate_cohens_h(observed_acc, pcc)
multiplier  <- round(observed_acc/pcc*100, 2)
power <- pwr.p.test(h = h, sig.level = fwer_alpha, alternative = "greater", n = nrow(df_res))$power
print(paste0('Model performs ', multiplier, '% better than PCC with ', power*100, '% power.'))

In [0]:
# Get PCC from apriori power analysis
var = 'Casualties - 5 sentence'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
pcc <- calculate_pcc(cat_counts)

# Get p from observed model accuracy
sheet_name <- "8. Casualties - 5 Sentence"
content <- dkuManagedFolderDownloadPath("m74dUoOQ","human_validation.xlsx", as="raw")
temp_f=tempfile()
writeBin(object=content, con=temp_f)
df_res <- read.xlsx(temp_f, sheetName=sheet_name)
colnames(df_res) <- c('analysis_id', 'ai_rating', 'human_rating', 'match')
observed_acc <- df_res %>% drop_na() %>% pull(match) %>% mean()

# Check power if at least 80%
h <- calculate_cohens_h(observed_acc, pcc)
multiplier  <- round(observed_acc/pcc*100, 2)
power <- pwr.p.test(h = h, sig.level = fwer_alpha, alternative = "greater", n = nrow(df_res))$power
print(paste0('Model performs ', multiplier, '% better than PCC with ', power*100, '% power.'))

In [0]:
# Get PCC from apriori power analysis
var = 'War Crimes Allegation Presence?'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
pcc <- calculate_pcc(cat_counts)

# Get p from observed model accuracy
sheet_name <- "9. War Crimes Allegation Presen"
content <- dkuManagedFolderDownloadPath("m74dUoOQ","human_validation.xlsx", as="raw")
temp_f=tempfile()
writeBin(object=content, con=temp_f)
df_res <- read.xlsx(temp_f, sheetName=sheet_name)
colnames(df_res) <- c('analysis_id', 'ai_rating', 'human_rating', 'match')
observed_acc <- df_res %>% drop_na() %>% pull(match) %>% mean()

# Check power if at least 80%
h <- calculate_cohens_h(observed_acc, pcc)
multiplier  <- round(observed_acc/pcc*100, 2)
power <- pwr.p.test(h = h, sig.level = fwer_alpha, alternative = "greater", n = nrow(df_res))$power
print(paste0('Model performs ', multiplier, '% better than PCC with ', power*100, '% power.'))

In [0]:
# Get PCC from apriori power analysis
var = 'War Crimes Perpetrator Mentioned?'
cat_1 <- df %>% filter(name == var) %>% pull(cat_1)
cat_2 <- df %>% filter(name == var) %>% pull(cat_2)
cat_counts <- c(cat_1, cat_2)
pcc <- calculate_pcc(cat_counts)

# Get p from observed model accuracy
sheet_name <- "10. War Crimes Perpetrator Ment"
content <- dkuManagedFolderDownloadPath("m74dUoOQ","human_validation.xlsx", as="raw")
temp_f=tempfile()
writeBin(object=content, con=temp_f)
df_res <- read.xlsx(temp_f, sheetName=sheet_name)
colnames(df_res) <- c('analysis_id', 'ai_rating', 'human_rating', 'match')
observed_acc <- df_res %>% drop_na() %>% pull(match) %>% mean()

# Check power if at least 80%
h <- calculate_cohens_h(observed_acc, pcc)
multiplier  <- round(observed_acc/pcc*100, 2)
power <- pwr.p.test(h = h, sig.level = fwer_alpha, alternative = "greater", n = nrow(df_res))$power
print(paste0('Model performs ', multiplier, '% better than PCC with ', power*100, '% power.'))